In [1]:
%pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install xgboost

   ---------------------------------------- 99.8/99.8 MB 778.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd

from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score,train_test_split,GridSearchCV

from xgboost import XGBClassifier

from sklearn.metrics import classification_report,roc_auc_score

In [3]:
df = pd.read_csv('../stroke_data_cleaned.csv')
df.head()

,Unnamed: 0,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,bmi_gr,health_status,age_gr
0,0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1,Extremely Obese,hd_only,Old
1,1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.893237,never smoked,1,Overweight,No disease,Old
2,2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1,Obese,hd_only,Old
3,3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1,Obese,No disease,Mature Adult
4,4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1,Normal,ht only,Old


In [4]:
df.isnull().sum()

Unnamed: 0           0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
bmi_gr               0
health_status        0
age_gr               0
dtype: int64

In [6]:
df2 = df[df.columns[1:12]]
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   object 
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int64  
 3   heart_disease      5110 non-null   int64  
 4   ever_married       5110 non-null   object 
 5   work_type          5110 non-null   object 
 6   Residence_type     5110 non-null   object 
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                5110 non-null   float64
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 439.3+ KB


In [7]:
gender_labels = {'Male':0,'Female':1}
ever_married_labels = {'Yes':0,'No':1}
work_type_labels = {'Private' : 0, 'Self-employed' : 1, 'Govt_job' : 2, 'children' : 3, 'Never_worked' : 4}
residence_type_labels = {'Urban' : 0, 'Rural' : 1}
smoking_status_labels = {'formerly smoked' : 0, 'never smoked' : 1, 'smokes' : 2, 'Unknown' : 3}

df2['gender'] = df2['gender'].map(gender_labels)
df2['ever_married'] = df2['ever_married'].map(ever_married_labels)
df2['work_type'] = df2['work_type'].map(work_type_labels)
df2['Residence_type'] = df2['Residence_type'].map(residence_type_labels)
df2['smoking_status'] = df2['smoking_status'].map(smoking_status_labels)

C:\Users\harsh\AppData\Local\Temp/ipykernel_1880/356677554.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['gender'] = df2['gender'].map(gender_labels)
C:\Users\harsh\AppData\Local\Temp/ipykernel_1880/356677554.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ever_married'] = df2['ever_married'].map(ever_married_labels)
C:\Users\harsh\AppData\Local\Temp/ipykernel_1880/356677554.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [8]:
X = df2.drop(columns=['stroke']).values
y = df2['stroke'].values

In [9]:
X

array([[  0.        ,  67.        ,   0.        , ..., 228.69      ,
         36.6       ,   0.        ],
       [  1.        ,  61.        ,   0.        , ..., 202.21      ,
         28.89323691,   1.        ],
       [  0.        ,  80.        ,   0.        , ..., 105.92      ,
         32.5       ,   1.        ],
       ...,
       [  1.        ,  35.        ,   0.        , ...,  82.99      ,
         30.6       ,   1.        ],
       [  0.        ,  51.        ,   0.        , ..., 166.29      ,
         25.6       ,   0.        ],
       [  1.        ,  44.        ,   0.        , ...,  85.28      ,
         26.2       ,   3.        ]])

In [10]:
y

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [11]:
X = MinMaxScaler().fit_transform(X)

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y.ravel(),test_size=0.3,random_state=54)

In [14]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=2)
x_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(x_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

Before OverSampling, counts of label '1': 171
Before OverSampling, counts of label '0': 3406 

After OverSampling, the shape of train_X: (6812, 10)
After OverSampling, the shape of train_y: (6812,) 

After OverSampling, counts of label '1': 3406
After OverSampling, counts of label '0': 3406


In [15]:
classifier = XGBClassifier(eval_metric='error')
classifier.fit(x_train_res,y_train_res)
y_pred = classifier.predict(X_test)

In [16]:
accuracies = cross_val_score(
                estimator = classifier,
                X = x_train_res,
                y = y_train_res,
                scoring='accuracy',
                cv = 10
            )

print('classification report:')
print(classification_report(y_test,y_pred))
print('roc_auc_score:')
print(roc_auc_score(y_test,y_pred))
print('cross validation mean accuracy:')
print(accuracies.mean()*100)

classification report:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      1455
           1       0.19      0.13      0.15        78

    accuracy                           0.93      1533
   macro avg       0.57      0.55      0.56      1533
weighted avg       0.92      0.93      0.92      1533

roc_auc_score:
0.5496695744118424
cross validation mean accuracy:
95.37757136520814


In [17]:
param_grid = {
    'learning_rate':[0.01, 0.05, 0.1,0.15,0.2],
    'max_depth':[4,5,6],
    'gamma':[0.0,0.1,0.2,0.3]
}

In [22]:
grid_search = GridSearchCV(
                estimator= classifier,
                param_grid = param_grid,
                scoring = 'roc_auc',
                cv = 10
            )

grid_search.fit(x_train_res,y_train_res)

print('best score:',grid_search.best_score_)
print('best params:',grid_search.best_params_)

c:\Users\harsh\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py:160: UserWarning: [10:35:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "cv", "scoring" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\harsh\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py:160: UserWarning: [10:35:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "cv", "scoring" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\harsh\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py:160: UserWarning: [10:35:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "cv", "scoring" } are not used.

  w

best score: 0.9910790190294607
best params: {'gamma': 0.0, 'learning_rate': 0.2, 'max_depth': 6}


In [23]:
classifier = XGBClassifier(
                objective = 'binary:logistic',
                learning_rate = 0.2,
                max_depth = 6,
                gamma = 0.0,
                scoring = 'roc_auc',
                eval_metric = 'error',
                cv = 10
            )

classifier.fit(x_train_res,y_train_res)
y_pred = classifier.predict(X_test)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.96      0.96      1455
           1       0.14      0.12      0.13        78

    accuracy                           0.92      1533
   macro avg       0.55      0.54      0.54      1533
weighted avg       0.91      0.92      0.92      1533



In [24]:
grid_search2 = GridSearchCV(
                estimator= classifier,
                param_grid = param_grid,
                scoring = 'accuracy',
                cv = 10
            )

grid_search2.fit(x_train_res,y_train_res)

print('best score:',grid_search2.best_score_)
print('best params:',grid_search2.best_params_)

c:\Users\harsh\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py:160: UserWarning: [10:38:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "cv", "scoring" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\harsh\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py:160: UserWarning: [10:38:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "cv", "scoring" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\harsh\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py:160: UserWarning: [10:38:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "cv", "scoring" } are not used.

  w

best score: 0.9505447397091565
best params: {'gamma': 0.1, 'learning_rate': 0.2, 'max_depth': 6}


In [25]:
grid_search3 = GridSearchCV(
                estimator= classifier,
                param_grid = param_grid,
                scoring = 'f1',
                cv = 10
            )

grid_search3.fit(x_train_res,y_train_res)

print('best score:',grid_search3.best_score_)
print('best params:',grid_search3.best_params_)

c:\Users\harsh\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py:160: UserWarning: [10:41:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "cv", "scoring" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\harsh\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py:160: UserWarning: [10:41:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "cv", "scoring" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\harsh\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py:160: UserWarning: [10:41:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "cv", "scoring" } are not used.

  w

best score: 0.9481688826428819
best params: {'gamma': 0.1, 'learning_rate': 0.2, 'max_depth': 6}


In [26]:
grid_search4 = GridSearchCV(
                estimator= classifier,
                param_grid = param_grid,
                scoring = 'recall',
                cv = 10
            )

grid_search4.fit(x_train_res,y_train_res)

print('best score:',grid_search4.best_score_)
print('best params:',grid_search4.best_params_)

c:\Users\harsh\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py:160: UserWarning: [10:44:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "cv", "scoring" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\harsh\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py:160: UserWarning: [10:44:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "cv", "scoring" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\harsh\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py:160: UserWarning: [10:44:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "cv", "scoring" } are not used.

  w

best score: 0.9498266344661033
best params: {'gamma': 0.0, 'learning_rate': 0.2, 'max_depth': 6}
